In [6]:
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import random
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import sklearn
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
import math
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.preprocessing import minmax_scale
from numpy import std, mean
import statistics
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector


#Import File
global all_df
all_df=pd.read_csv('TrainDataset2023.csv', index_col=False) #Read from File
all_df.drop('ID', axis=1, inplace=True) # Drop ID - not needed for training

#Impute Missing Values
imputer = SimpleImputer(missing_values = 999, strategy="median") # Test Other Methods
SimpleImputer(missing_values = 999)
s = 0
for i in all_df:
    imputer.fit(all_df)
    array = np.array(all_df[i])
    all_df[i] = imputer.fit_transform(array.reshape(-1, 1))       
    
#Normalise Values

#Min Max 
colno = 0
for col in all_df:
    if col == 'RelapseFreeSurvival (outcome)':
        continue
    if colno >= 12: # ONLY NORMALISE MRI SCAN DATA
        colmean = np.median(all_df[col])
        colstd = np.std(all_df[col])
        upper = colmean + (3*colstd)
        lower = colmean - (3*colstd) # USING MIN ALSO SEEMS TO SKEW DATA
        #Comment out to cancel
        #all_df[col] = minmax_scale(all_df[col], feature_range=(lower,upper)) #Minimal Change - Downscales severity of Mean Squared Error
    colno+=1
        
#Z Score
#scaler = StandardScaler()
#for col in all_df:
#    scaler.fit(all_df[[col]])
#    scaler.mean_
#    all_df[col] = scaler.transform(all_df[[col]]) # Still seeing minimal change
    
#Things to Consider
# Curse of Dimensionality - We have fewer data points, but a large selection of features available
    
# Data Normilisation Methods - To Do/Consider
# 1) Z-Normilisation - DONE - Trying to avoid scaling
# 2) Min-Max Normilisation - DONE - using STD to avoid major changes (most outliers are acceptable)
# 3) Vector Normilisation - Ad Hoc
    
#Feature Selection Methods
# 1) Forward Feature Selection - Done
# 2) LASSO - Regression Method - Done
# 3) Chi Square - Categorical
# 4) T-test - Categorical
# 5) ANOVA - Categorical
# 6) Recursive Feature Elimination - Very Expensive - Not loading in reasonable period (Output Not Tested)
# 7) Sequential Feature Selection
# 8) Ridge/ElasticNet
# 9) Correlation/Variance

#Dimensionality Reduction Methods
# 1) PCA (Principle Component Analysis)
# 2) LDA (Linear Discriminant Analysis)

# Regression Methods
# 1) Linear Regression - Done (cleanup)
# 2) SVM - Done
# 3) Decision Tree - Done
# 4) Random Forest - Done (needs refining)
# 5) ANN - Done

# Evaluation Methods
# 1) K-Fold
print('Data Normilisation Complete')

Data Normilisation Complete


In [11]:
#Linear Regression - High MSE by default
print('Linear Regression Start \n')
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
lin_reg = LinearRegression()

#Forward Feature Selection
# LIMIT HOW MANY FEATURES CAN BE USED?
def FFS(trainx, trainy, validationx, validationy, maxscore):
    F = trainx.columns.tolist()
    #F.remove('RelapseFreeSurvival (outcome)')
    X = []
    B = maxscore
    while X != F:
        Y = []
        remfeatures = list(set(F)-set(Y))
        for i in remfeatures:
            temp = Y+[i]
            lin_reg.fit(trainx[temp],trainy)
            lin_reg.intercept_,lin_reg.coef_
            y_pred = lin_reg.predict(validationx[temp])
            mse = mean_absolute_error(validationy, y_pred)
            if mse <= B-(B/40): #Fine Tune the Optimal Increase for a new feature to be worthwhile
                B = mean_absolute_error(validationy, y_pred)
                Y = temp
                #print("Features Updated: "+str(Y))
                #print("Best Score Updated: "+str(B))
        if X != [] and mean_absolute_error(validationy, y_pred) < B:
            #print("Best X Features = "+str(X))
            #print("Best Features Accuracy = "+str(B))
            break
        else:
            X = Y
            break
    print("Best Features = "+str(X))
    print('Number of Features Used = '+str(len(X)))
    #y_pred = lin_reg.predict(validationx[X])
    #print("Feature Mean Squared Error = "+str(mean_squared_error(validationy, y_pred)))
    print("")
    return X

def FFS2(xset, yset, maxscore):
    F = xset.columns.tolist()
    #F.remove('RelapseFreeSurvival (outcome)')
    X = []
    B = maxscore
    while X != F: #Add Limit on how many features can be used
        Y = []
        remfeatures = list(set(F)-set(Y))
        for i in remfeatures:
            temp = Y+[i]
            lin_reg.fit(xset[temp],yset)
            lin_reg.intercept_,lin_reg.coef_
            y_pred = lin_reg.predict(xset[temp])
            mse = mean_absolute_error(yset, y_pred)
            if mse <= B-(B/40):
                B = mean_absolute_error(yset, y_pred)
                Y = temp
                #print("Features Updated: "+str(Y))
                #print("Best Score Updated: "+str(B))
        if X != [] and mean_absolute_error(yset, y_pred) < B:
            #print("Best X Features = "+str(X))
            #print("Best Features Accuracy = "+str(B))
            break
        else:
            X = Y
            break
    print("Best Features = "+str(X))
    print('Number of Features Used = '+str(len(X)))
    #y_pred = lin_reg.predict(validationx[X])
    #print("Feature Mean Squared Error = "+str(mean_squared_error(validationy, y_pred)))
    print("")
    return X

# Add validation Set for testing FFS

X = FFS(train_X, train_y, validate_X, validate_y, 1000) #Test FFS
scaler = StandardScaler()
Xs = scaler.fit_transform(train_X[X]) # Change when adding validation set
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    lin_reg.fit(Xs[train],y[train])
    lin_reg.intercept_,lin_reg.coef_
    regression_score = lin_reg.score(Xs[test], y[test])
    y_pred = lin_reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = lin_reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print(mse)
    print(mse2)
    print("")
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2

print('Validation Set: ')
print('The Avg. Accuracy of Linear Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Linear Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Linear Regression Train is {:03.4f}'.format(mse2_total/5))
print("")

X = FFS2(x, y, 1000) #Test FFS
scaler = StandardScaler()
Xs = scaler.fit_transform(all_df[X]) # Change when adding validation set
mse_total = 0
mse2_total
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    lin_reg.fit(Xs[train],y[train])
    lin_reg.intercept_,lin_reg.coef_
    regression_score = lin_reg.score(Xs[test], y[test])
    y_pred = lin_reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = lin_reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print(mse)
    print(mse2)
    print("")
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2

print('\nNo Validation Set: ')
print('The Avg. Accuracy of Linear Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Linear Regression Test is {:03.4f}'.format(mse_total/5))
print('The Avg. Mean Square Error for Linear Regression Train is {:03.4f}'.format(mse2_total/5))
print('')

#Create loop to determine best no of features
featureno = 1
score = 10000
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
while featureno < 50:
    featuretest = []
    sfs = SequentialFeatureSelector(lin_reg, n_features_to_select=featureno)
    sfs.fit(x,y)
    feats = sfs.get_feature_names_out() 
    for i in feats:
        featuretest.append(i)
    x2 = all_df[featuretest] 
    lin_reg.fit(x2,y)
    lin_reg.intercept_,lin_reg.coef_
    y_pred = lin_reg.predict(x2)
    mse = mean_absolute_error(y, y_pred)
    if mse < score-(score/40):
        featureno+=1
        score = mse
        continue
    else:
        break
    
print("SFS Best No. of Features = "+str(featureno))
    
sfs = SequentialFeatureSelector(lin_reg, n_features_to_select=featureno)
sfs.fit(x,y)
feats = sfs.get_feature_names_out() 
print(feats)
feature = []
for i in feats:
    feature.append(i)

y = all_df['RelapseFreeSurvival (outcome)']
x = all_df[feature]
Xs = scaler.fit_transform(x) 
mse_total = 0
mse2_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    lin_reg.fit(Xs[train],y[train])
    lin_reg.intercept_,lin_reg.coef_
    regression_score = lin_reg.score(Xs[test], y[test])
    y_pred = lin_reg.predict(Xs[test])
    mse = mean_absolute_error(y[test], y_pred)
    y_pred2 = lin_reg.predict(Xs[train])
    mse2 = mean_absolute_error(y[train], y_pred2)
    print(mse)
    print(mse2)
    print("")
    score_total += regression_score
    mse_total += mse
    mse2_total += mse2

print('\nSFS Set: ')
print('The Avg. Accuracy of Linear Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Linear Regression Test is {:03.4f}'.format(mse_total/5))  
print('The Avg. Mean Square Error for Linear Regression Train is {:03.4f}'.format(mse2_total/5))

print('\nLinear Regression End \n')

Linear Regression Start 

Best Features = ['original_glcm_SumAverage', 'original_shape_MajorAxisLength', 'HER2', 'original_glszm_ZonePercentage', 'original_shape_Sphericity', 'original_glcm_JointAverage']
Number of Features Used = 6

46.23166846375699
16.736654682423246

24.659063772843915
22.17221399757321

13.223831686633982
25.064825822546478

15.959133827991419
24.622926417576362

25.471112265739453
22.818609835404946

Validation Set: 
The Avg. Accuracy of Linear Regression is -1.23
The Avg. Mean Square Error for Linear Regression Test is 25.1090
The Avg. Mean Square Error for Linear Regression Train is 22.2830

Best Features = ['original_glcm_SumAverage']
Number of Features Used = 1

43.68102060542206
15.959511261456308

21.099174101272165
21.199772212011865

11.680622629403643
23.710283670850618

24.408664290152206
21.061809221773466

13.360499999966828
23.15928108300813


No Validation Set: 
The Avg. Accuracy of Linear Regression is -0.84
The Avg. Mean Square Error for Linear Re

In [22]:
# SVM Regressor Start
print('\nSVM Regression Start \n')
y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
#Forward Feature Selection for 1 feature
F = all_df.columns.tolist()
F.remove('RelapseFreeSurvival (outcome)')
B = 1000
Y = []
X = []
regressor = SVR(kernel = 'rbf')
columnno = 0
for i in F:
    X = train_X[[i]].values
    regressor.fit(X, train_y.values)
    y_pred = regressor.predict(validate_X[[i]])
    mse = mean_squared_error(validate_y, y_pred)
    if mse < B:
        Y = i
        B = mse
        #print("Best Feature Updated: "+str(i))
        #print("Best Accuracy Updated: "+str(B))

print('SVM Best Feature = '+str(Y))   

#Feature Elimination
#regressor2 = SVR(kernel = 'linear')
#selector = RFE(regressor2, n_features_to_select=1)
#selector = selector.fit(x,y)
#print(selector.ranking_)
#print('RFE = '+str(selector.estimator_))


#Sequential Feature Selection
#sfs = SequentialFeatureSelector(regressor, n_features_to_select=1)
#sfs.fit(x,y)
#feats = sfs.get_feature_names_out() 
#print(feats)



#Set Best Feature
x = train_X[[Y]].values

#tmp x for SFS
#x = train_X[[feats[0]]].values

Xs = scaler.fit_transform(x)

#K-fold
mse_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, train_y):
    regressor.fit(Xs[train],y[train])
    regression_score = regressor.score(Xs[test], y[test])
    y_pred = regressor.predict(Xs[test])
    mse = mean_squared_error(y[test], y_pred)
    print(mse)
    score_total += regression_score
    mse_total += mse

print('The Avg. Accuracy of SVM Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for SVM Regression is {:03.4f}'.format(mse_total/5))
print('\nSVM Regression End \n')


SVM Regression Start 

SVM Best Feature = original_glszm_ZoneEntropy
['original_shape_Sphericity']
2695.5080701112984
880.8462296610608
321.7956102827304
496.97687552784896
564.5021345309962
The Avg. Accuracy of SVM Regression is -1.08
The Avg. Mean Square Error for SVM Regression is 991.9258

SVM Regression End 



In [24]:
# Decision Tree Regression Start
print('\nDecision Tree Regression Start \n')

x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
scaler = StandardScaler()
Xs = scaler.fit_transform(x)

depth = 1
top_score = 10000
best_depth = 1
train_X, validate_X, train_y, validate_y = train_test_split(x, y, test_size=0.15, shuffle = False)
while depth <= 50:
    tree_clf = DecisionTreeRegressor(max_depth=depth)
    tree_clf.fit(train_X, train_y) #Train Model
    tree_clf.predict(train_X)
    new_score = tree_clf.score(validate_X, validate_y)
    y_pred = tree_clf.predict(validate_X)
    mse = mean_squared_error(validate_y, y_pred)
    #print("Depth = "+str(depth)+", MSE = "+str(mse))
    if mse <= top_score:
        best_depth = depth
        top_score = mse
        #print("Depth = "+str(depth)+", Score = "+str(new_score))
    depth += 1
    
print("Best Depth = "+str(best_depth))

#K-fold
mse_total = 0
score_total = 0
kf = KFold(n_splits=5)
tree_clf = DecisionTreeRegressor(max_depth=best_depth)
for train, test in kf.split(train_X, train_y):
    tree_clf.fit(Xs[train],y[train])
    regression_score = tree_clf.score(Xs[test], y[test])
    y_pred = tree_clf.predict(Xs[test])
    mse = mean_squared_error(y[test], y_pred)
    print(mse)
    score_total += regression_score
    mse_total += mse

print('The Avg. Accuracy of Decision Tree Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Decision Tree Regression is {:03.4f}'.format(mse_total/5))
print('\nDecision Tree Regression End \n')
# Decision Tree Regression End



Decision Tree Regression Start 

Best Depth = 1
2545.027346355209
847.9014682274044
367.4413178292528
497.69985487520887
651.1109299801504
The Avg. Accuracy of Decision Tree Regression is -1.19
The Avg. Mean Square Error for Decision Tree Regression is 981.8362

Decision Tree Regression End 



In [25]:
#Neural Network Regression Start
print('\nNeural Network Regression Start \n')

X = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

#K-fold
mse_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    mlp_clf = MLPRegressor(random_state=1, max_iter=300).fit(Xs[train], y[train])
    regression_score = mlp_clf.score(Xs[test], y[test])
    y_pred = mlp_clf.predict(Xs[test])
    mse = mean_squared_error(y[test], y_pred)
    print(mse)
    print(mlp_clf.n_layers_)
    score_total += regression_score
    mse_total += mse

print('The Avg. Accuracy of Neural Network Regression is {:03.2f}'.format(score_total/5))
print('The Avg. Mean Square Error for Neural Network Regression is {:03.4f}'.format(mse_total/5))
print('\nNeural Network Regression End \n')
# Neural Network Regression End


Neural Network Regression Start 

2235.0422657305176
3
585.7467784979947
3
518.759685845668
3
623.0502159359238
3
616.4603620388452
3
The Avg. Accuracy of Neural Network Regression is -1.15
The Avg. Mean Square Error for Neural Network Regression is 915.8119

Neural Network Regression End 



In [37]:
#Random Forest Regression Start
print('\nRandom Forest Regression Start \n')

x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
y = all_df['RelapseFreeSurvival (outcome)']

#Find best feature to accuracy ratio - random forest has built in feature selection via feature ranking
# return Highest rated features
def RFDepthSelection(trainx, trainy, testx, testy):
    feature_limit = 1
    mse_score = 1000
    while feature_limit < 50:
        rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = feature_limit)
        rfregressor.fit(trainx, trainy)
        y_pred = rfregressor.predict(testx)
        mse = mean_squared_error(testy, y_pred)
        if mse < mse_score - (mse_score/20):
            mse_score = mse
            feature_limit+=1
            continue
        else:
            return feature_limit

#Make x the validated feature
#rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = feature_limit) #Default Measure = MSE
#K-fold
Xs = scaler.fit_transform(x)
mse_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    featureno = RFDepthSelection(Xs[train], y[train], Xs[test], y[test]) #Use Nested Kfold for this?
    print(featureno)
    rfregressor = RandomForestRegressor(n_estimators=100, random_state = 0, max_features = featureno)
    rfregressor.fit(Xs[train], y[train])
    y_pred = rfregressor.predict(Xs[test])
    mse = mean_squared_error(y[test], y_pred)
    print(mse)
    score_total += regression_score
    mse_total += mse

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for Random Forest Regression is {:03.4f}'.format(mse_total/5))
print('\nRandom Forest Regression End \n')

#Random Forest Regression End


Random Forest Regression Start 

1
2309.7383068216604
2
634.0435609015101
2
341.0219651727146
2
670.1780929000224
2
295.72777123917916
The Avg. Mean Square Error for Random Forest Regression is 850.1419

Random Forest Regression End 



In [1]:
print('\nLASSO Linear Regression Start \n')

y = all_df['RelapseFreeSurvival (outcome)']
x = all_df.drop('RelapseFreeSurvival (outcome)', axis=1)
clf = linear_model.Lasso()

Xs = scaler.fit_transform(x)
mse_total = 0
score_total = 0
kf = KFold(n_splits=5)
for train, test in kf.split(Xs, y):
    clf.fit(Xs[train],y[train])
    y_pred = clf.predict(Xs[test])
    mse = mean_squared_error(y[test], y_pred)
    print(mse)
    score_total += regression_score
    mse_total += mse

#print("Best Number of Features = "+str(feature_limit))
print('The Avg. Mean Square Error for LASSO Regression is {:03.4f}'.format(mse_total/5))
print('\nLASSO Linear Regression End \n')


LASSO Linear Regression Start 



NameError: name 'all_df' is not defined